## 9.5 텐서플로 함수 활용하기  
새로운 구능 기현시 유연하게 대처하는 방법으로 텐서플로와 케라스를 섞어 쓰는 방법을 다룹니다.

In [1]:
# set to use CPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['KERAS_BACKEND'] = 'tensorflow'
#os.environ['KERAS_BACKEND'] = 'theano'

### 9.5.1 텐서플로와 케라스 패키지 임포트 및 상호 연결

In [2]:
import tensorflow as tf
print(tf.keras.__version__)

'2.3.0-tf'

### 9.5.2 완전 연결층 인공지능망 모델링

In [3]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import categorical_accuracy, categorical_crossentropy

class DNN(Model):
    def __init__(self, Nin=2, Nh_l=[2,2], Nout=2):
        super(DNN, self).__init__()
        self.dense1 = Dense(Nh_l[0], activation='relu')
        self.dense2 = Dense(Nh_l[1], activation='relu')
        self.dense3 = Dense(Nout, activation='softmax')
        
    def call(self, x):
        x = self.dense1(x)
        x = Dropout(0.5)(x)
        x = self.dense2(x)
        x = Dropout(0.25)(x)
        return self.dense3(x)

### 9.5.3 데이터 준비 단계

In [4]:
import numpy as np
from keras import datasets  # mnist
from keras.utils import np_utils  # to_categorical

def Data_func():
    (X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

    #Y_train = np_utils.to_categorical(y_train)
    #Y_test = np_utils.to_categorical(y_test)

    L, H, W = X_train.shape
    X_train = X_train.reshape(-1, H * W)
    X_test = X_test.reshape(-1, H * W)

    X_train = (X_train / 255.0).astype(np.float32)
    X_test = (X_test / 255.0).astype(np.float32)

    return (X_train, y_train), (X_test, y_test)

## 9.5.4 학습 진행 및 효과 분석 단계

In [5]:
from keraspp.skeras import plot_loss, plot_acc
import matplotlib.pyplot as plt

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#  Nin=2, Nh_l=[2,2], Nout=2)
Nin = 784
Nh_l = [100, 50]
number_of_class = 10
Nout = number_of_class

data = Data_func()
model = DNN(Nin, Nh_l, Nout)
batch_size = 100

(X_train, Y_train), (X_test, Y_test) = data
X_train.shape, Y_train.shape

N_tr = X_train.shape[0]
epochs = 10
for epoch in range(epochs):
    for b in range(N_tr // batch_size):
        X_tr_b = X_train[batch_size * (b-1):batch_size * b]
        Y_tr_b = Y_train[batch_size * (b-1):batch_size * b]
        with tf.GradientTape() as tape:
            predictions = model(X_tr_b, training=True)
            loss = loss_object(Y_tr_b, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)
        train_accuracy(Y_tr_b, predictions)
        
    predictions = model(X_test, training=False)
    t_loss = loss_object(Y_test, predictions)
    test_loss(t_loss)
    test_accuracy(Y_test, predictions)
        
    print(
        f'Epoch {epoch}, '
        f'Loss: {train_loss.result():.3}, '
        f'Accuracy: {train_accuracy.result() * 100:.3}, '
        f'Test Loss: {test_loss.result():.3}, '
        f'Test Accuracy: {test_accuracy.result() * 100:.3}')

2021-11-09 22:17:30.061265: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-09 22:17:30.329915: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-09 22:17:30.329953: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SJKIM-Home): /proc/driver/nvidia/version does not exist
2021-11-09 22:17:30.331198: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-11-09 22:17:30.349244: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3599995000 Hz
2021-11-09 22:17:30.350895: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5637454852e0 initialized for platform Host (this does not guarantee that XLA will be used). Devi

Epoch 0, Loss: 1.71, Accuracy: 77.7, Test Loss: 1.55, Test Accuracy: 91.4
Epoch 1, Loss: 1.64, Accuracy: 83.1, Test Loss: 1.54, Test Accuracy: 92.0
Epoch 2, Loss: 1.62, Accuracy: 85.4, Test Loss: 1.54, Test Accuracy: 92.6
Epoch 3, Loss: 1.6, Accuracy: 86.9, Test Loss: 1.53, Test Accuracy: 92.9
Epoch 4, Loss: 1.59, Accuracy: 87.9, Test Loss: 1.53, Test Accuracy: 93.2
Epoch 5, Loss: 1.58, Accuracy: 88.6, Test Loss: 1.53, Test Accuracy: 93.5
Epoch 6, Loss: 1.57, Accuracy: 89.2, Test Loss: 1.53, Test Accuracy: 93.7
Epoch 7, Loss: 1.57, Accuracy: 89.6, Test Loss: 1.52, Test Accuracy: 93.9
Epoch 8, Loss: 1.56, Accuracy: 90.0, Test Loss: 1.52, Test Accuracy: 94.1
Epoch 9, Loss: 1.56, Accuracy: 90.3, Test Loss: 1.52, Test Accuracy: 94.2


---
### 9.2.7 전체 코드